# Apply a naive beta cooling

In [1]:
import numpy as np
from regressor import LinearRegressor

import inversion_ideas as ii

## Synthetic data

In [2]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

array([0.78225148, 0.67148671, 0.2373809 , 0.17946133, 0.34662367,
       0.15210999, 0.31142952, 0.23900652, 0.54355731, 0.91770851])

In [3]:
# Build the X matrix
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [4]:
# Generate synthetic data with noise
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
std_err =  1e-2 * maxabs
noise = rng.normal(scale=std_err, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

array([2.83840696, 2.18091081, 2.00623242, 2.08333039, 2.01694883,
       2.7826232 , 2.10564027, 1.27333506, 2.08859855, 1.94177648,
       1.88492037, 2.92394733, 2.17231952, 3.08009275, 1.61670886,
       1.77403753, 2.67305005, 1.91413882, 2.42117827, 2.13991628,
       2.0153805 , 2.71388471, 2.65944255, 2.44416121, 3.14217523])

## Naive inversion with beta scheduling

In [5]:
simulation = LinearRegressor(X)

In [6]:
uncertainty = std_err * np.ones_like(synthetic_data)
data_misfit = ii.DataMisfit(synthetic_data, uncertainty, simulation)
data_misfit

φd(m)

In [7]:
smallness = ii.TikhonovZero(n_params)
smallness

φs(m)

In [8]:
beta_0 = 1e4
regularization = beta_0 * smallness
regularization

1.00e+04 φs(m)

In [9]:
phi = data_misfit + regularization
phi

φd(m) + 1.00e+04 φs(m)

In [10]:
minimizer = ii.ConjugateGradient()

In [11]:
initial_model = np.zeros(n_params)

In [12]:
model = initial_model.copy()

chi_target = 1.0
beta_cooling_factor = 2.0

while True:
    # Minimize the objective function
    model = minimizer(phi, model) 

    # Stopping criteria: chi factor
    dmisfit = data_misfit(model)
    chi_factor = dmisfit / data_misfit.n_data
    if chi_factor <= chi_target:
        break

    # Cool beta
    regularization.multiplier /= beta_cooling_factor


inverted_model = model.copy()

In [13]:
inverted_model

array([0.78781238, 0.63620172, 0.2461426 , 0.1941919 , 0.33960904,
       0.17601471, 0.32485498, 0.26662799, 0.52718998, 0.89311305])

In [14]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.78781238 0.63620172 0.2461426  0.1941919  0.33960904 0.17601471
 0.32485498 0.26662799 0.52718998 0.89311305]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]
